Import Libraries

In [119]:
import pandas as pd
import numpy as np

Mount Drive

In [120]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [121]:
%cd /content/gdrive/MyDrive/Utility-Scoring-Of-Product-Reviews/Data

/content/gdrive/MyDrive/Utility-Scoring-Of-Product-Reviews/Data


Read dataset obtained from previous step where semantic scores are calculated

In [122]:
df=pd.read_csv("Cell_Phones_and_Accessories.csv")

In [123]:
df.head(4)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,preProcessed_reviewText,subjectivity_score,flesch_index,polarity_score,entropy_score,daleChall_index,helpful_ratio,lexDiversity_ratio,normalized_flesch_index,normalized_polarity_score,normalized_entropy_score,normalized_daleChall_index,utility_score
0,6073894996,"[3, 4]",4.0,This is a nice charger but you can tell it was...,"03 20, 2013",A29OXVQRZ154KX,this is a nice charger but you can tell it was...,0.650000,31.0,0.352083,5.632935,6.568111,0.666667,0.059278,0.013262,0.676042,0.384745,0.393515,0.428571
1,9861203192,"[25, 25]",5.0,"Bought this for my new AT&T; Galaxy SII, the c...","10 7, 2011",A1847XXVEA8QUK,bought this for my new att galaxy sii the case...,0.562121,34.1,0.266970,5.800280,7.892625,0.966667,0.055556,0.014807,0.633485,0.416500,0.477973,0.500000
2,9985537742,"[10, 11]",3.0,Works fine IF you don't move the phone. If yo...,"11 19, 2010",A2TOXBTDH3Y6R9,works fine if you dont move the phone if you ...,0.500000,25.1,0.280476,5.381458,7.162129,0.846154,0.068452,0.010320,0.640238,0.337025,0.431393,0.476190
3,998554627X,"[5, 5]",5.0,was somewhat concerned after reading some of t...,"02 23, 2013",A17X3JUWJY3AXG,was somewhat concerned after reading some of t...,0.575000,16.9,0.375000,5.064869,8.279561,0.857143,0.108696,0.006232,0.687500,0.276950,0.502646,0.500000


Creating list of parameters/variables

In [124]:
X_columns = ['subjectivity_score', 'normalized_flesch_index', 'normalized_polarity_score', 'normalized_entropy_score', 'normalized_daleChall_index', 'lexDiversity_ratio', 'helpful_ratio']

Segregating the columns

In [125]:
X = df[X_columns]
y = df['utility_score']

Splitting data into training and testing dataset

In [126]:
from sklearn.model_selection import train_test_split

In [127]:
x_train, x_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=1)

print("Train data points: ", x_train.shape[0])
print("test data points: ", x_test.shape[0])

Train data points:  8477
test data points:  2120


Tuning parameters(scoring is R Squared) of the models which gave the best results.

In [128]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score

In [129]:
from sklearn.model_selection import GridSearchCV

In [130]:
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor)
from sklearn.linear_model import LinearRegression, Ridge

List to store the results

In [131]:
Model = []
R_sq = []
cv = KFold(4)

Linear Regression

In [132]:
Model.append('LinearRegression')
LinearRegression()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [133]:
param_grid = {'fit_intercept': [True, False],
              'n_jobs': [0,5,10]}
clf = GridSearchCV(LinearRegression(), param_grid = param_grid, scoring='r2', cv=cv).fit(x_train, y_train)
print(clf.best_estimator_) 
tuned_r2 = clf.best_score_
print("R Squared:",tuned_r2)
R_sq.append(tuned_r2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=0, normalize=False)
R Squared: 0.5898014804209951


Ridge Regression

In [134]:
Model.append('Ridge')
Ridge()

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [135]:
param_grid = {'alpha': [0.5,1,1.5,2,2.5]}
clf = GridSearchCV(Ridge(random_state=1), param_grid = param_grid, scoring='r2', cv=cv).fit(x_train, y_train)
print(clf.best_estimator_) 
tuned_r2 = clf.best_score_
print("R Squared:",tuned_r2)
R_sq.append(tuned_r2)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=1, solver='auto', tol=0.001)
R Squared: 0.5898423218199768


Gradient Boosting Regressor

In [136]:
Model.append('Gradient Boosting Regressor')
GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [137]:
param_grid = {'n_estimators': range(100,200,50),
              'max_depth': range(2,4,2), 
              'min_samples_split': range(2,4,2), 
              'learning_rate':[0.1,0.2,0.3]}
clf = GridSearchCV(GradientBoostingRegressor(random_state=1), param_grid = param_grid, scoring='r2', cv=cv).fit(x_train, y_train)
print(clf.best_estimator_) 
tuned_r2 = clf.best_score_
print("R Squared:",tuned_r2)
R_sq.append(tuned_r2)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=1, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
R Squared: 0.6376057180532235


Random Forest Regressor

In [138]:
Model.append('Random Forest Regressor')
RandomForestRegressor()

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [139]:
param_grid = {'n_estimators': range(100,200,50),
              'max_depth': range(2,6,2), 
              'min_samples_split': range(2,6,2)}
clf = GridSearchCV(RandomForestRegressor(random_state=1), param_grid = param_grid, scoring='r2', cv=cv).fit(x_train, y_train)
print(clf.best_estimator_) 
tuned_r2 = clf.best_score_
print("R Squared:",tuned_r2)
R_sq.append(tuned_r2)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)
R Squared: 0.6317889290651129


Scores after tuning

In [140]:
tuningEvaluation = pd.DataFrame({'Model': Model,
                           'R Squared': R_sq})
print("Following are the scores after tuning: ")
tuningEvaluation

Following are the scores after tuning: 


,Model,R Squared
0,LinearRegression,0.589801
1,Ridge,0.589842
2,Gradient Boosting Regressor,0.637606
3,Random Forest Regressor,0.631789
